In [2]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision 
import torchvision.transforms as transforms
import time


In [3]:

# Preparing for Data
print('==> Preparing data..')

# Training Data augmentation
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
# Testing Data preparation
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

==> Preparing data..


In [4]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 6, 5)  # 3 input channels, 6 output channels, 5x5 kernel
        self.pool = nn.MaxPool2d(2, 2)  # 2x2 max pooling with stride 2
        self.conv2 = nn.Conv2d(6, 16, 5) # 6 input channels, 16 output channels, 5x5 kernel
        self.fc1 = nn.Linear(16 * 5 * 5, 120) # fully connected layer with 120 output units
        self.fc2 = nn.Linear(120, 84) # fully connected layer with 84 output units
        self.fc3 = nn.Linear(84, 10) # fully connected layer with 10 output units (for 10 classes)
        self.dropout1 = nn.Dropout(p=0.5) 
        self.dropout2 = nn.Dropout(p=0.5)
        self.dropout = True
        self.batch_norm = False

    def forward(self, x):
        
        x = self.pool(F.relu(self.conv1(x)))  # 1st conv layer, ReLU activation, max pooling
        x = self.pool(F.relu(self.conv2(x)))  # 2nd conv layer, ReLU activation, max pooling
        x = x.view(-1, 16 * 5 * 5)           # flatten output of conv layer
        x = F.relu(self.fc1(x))              # 1st fully connected layer, ReLU activation
        x = F.relu(self.fc2(x))              # 2nd fully connected layer, ReLU activation
        if self.dropout:                     # add dropout layer if enabled
            x = F.dropout(x, training=self.training)
        x = self.fc3(x)                      # output layer

        return x

def train(model, device, train_loader, optimizer, epoch):
    model.train()
    count = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test( model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


In [ ]:
def main():
    time0 = time.time()
    # Training settings
    batch_size = 128
    epochs = 20
    lr = 0.05
    no_cuda = False
    save_model = False
    use_cuda = not no_cuda and torch.cuda.is_available()
    torch.manual_seed(100)
    device = torch.device("cuda" if use_cuda else "cpu")
    
    trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
    train_loader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True)
    testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
    test_loader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False)

    model = LeNet().to(device)
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)

    for epoch in range(1, epochs + 1):
        train( model, device, train_loader, optimizer, epoch)
        test( model, device, test_loader)

    if (save_model):
        torch.save(model.state_dict(),"cifar_lenet.pt")
    time1 = time.time() 
    print ('Traning and Testing total excution time is: %s seconds ' % (time1-time0))   
if __name__ == '__main__':
    main()

100%|██████████| 170498071/170498071 [00:02<00:00, 69285508.10it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Train Epoch: 1 [0/50000 (0%)]	Loss: 2.313206
Train Epoch: 1 [1280/50000 (3%)]	Loss: 2.290324
Train Epoch: 1 [2560/50000 (5%)]	Loss: 2.256552
Train Epoch: 1 [3840/50000 (8%)]	Loss: 2.170691
Train Epoch: 1 [5120/50000 (10%)]	Loss: 2.070918
Train Epoch: 1 [6400/50000 (13%)]	Loss: 2.066700
Train Epoch: 1 [7680/50000 (15%)]	Loss: 2.069912
Train Epoch: 1 [8960/50000 (18%)]	Loss: 2.015686
Train Epoch: 1 [10240/50000 (20%)]	Loss: 1.980385
Train Epoch: 1 [11520/50000 (23%)]	Loss: 1.915211
Train Epoch: 1 [12800/50000 (26%)]	Loss: 1.879259
Train Epoch: 1 [14080/50000 (28%)]	Loss: 2.036569
Train Epoch: 1 [15360/50000 (31%)]	Loss: 1.935795
Train Epoch: 1 [16640/50000 (33%)]	Loss: 1.901657
Train Epoch: 1 [17920/50000 (36%)]	Loss: 1.942562
Train Epoch: 1 [19200/50000 (38%)]	Loss: 2.069175
Train Epoch: 1 [20480/50000 (41%)]	Loss: 1.872715
Train Epoch: 1 [21760/50000 (43%)]	Loss: 1.930039
Train Epoch: 1 [23040/5000